In [2]:
import pandas as pd
import checklist
from allennlp_models.pretrained import load_predictor
from allennlp_models.pretrained import get_pretrained_models
from checklist.editor import Editor
from checklist.perturb import Perturb
from checklist.test_types import MFT, INV, DIR
from checklist.expect import Expect
from checklist.pred_wrapper import PredictorWrapper

In [178]:
nsamples = 100
editor = Editor()

In [195]:
def make_list_shoot_passive(amount_per_format):
    shoot_passive_list = []
    shoot_passive = editor.template("{first_name1} is shot by {first_name2}.",
                                    meta=True, 
                                    nsamples=amount_per_format, 
                                    remove_duplicates=True, 
                                    seed=18)
    shoot_passive_list.append(shoot_passive['data'])
    shoot_passive_list = [x for xs in shoot_passive_list for x in xs]
    return shoot_passive_list

def make_list_shoot_passive_neg(amount_per_format):
    shoot_passive_list = []
    shoot_passive = editor.template("{first_name1} is not shot by {first_name2}.",
                                    meta=True, 
                                    nsamples=amount_per_format, 
                                    remove_duplicates=True, 
                                    seed=18)
    shoot_passive_list.append(shoot_passive['data'])
    shoot_passive_list = [x for xs in shoot_passive_list for x in xs]
    return shoot_passive_list

def get_tags_v0(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        tags = output['verbs'][0]['tags']
    else:
        tags = []
    return tags
        
def get_tags_v1(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        tags = output['verbs'][1]['tags']
    else:
        tags = []
    return tags
        
def get_descr_v0(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        descr = output['verbs'][0]['description']
    else:
        descr = []
    return descr

def get_descr_v1(sentence):
    output = srl_predictor.predict(sentence)
    if len(output['verbs']) >= 1:
        descr = output['verbs'][1]['description']
    else:
        descr = []
    return descr

def find_final_min_one_arg(list_of_arg):
    if list_of_arg:
        final = list_of_arg[-2]
    else:
        final = []
    return final

def find_first_arg(list_of_arg):
    if list_of_arg:
        first = list_of_arg[0]
    else:
        first = []
    return first
                                  
                                 
def passive_to_active(sentence):
    string_split = sentence.split()
    verb = string_split[2]
    if string_split[2] == 'shot':
        verb = 'shoots'
    
    if string_split[2] == 'killed':
        verb = 'kills'
        
    if string_split[2] == 'loved':
        verb = 'loves'
    
    arg0 = string_split[-1][:-1]
    arg1 = string_split[0]
    
    rebuilt_sentence = " ".join([arg0, verb, arg1])
    rebuilt_sentence = rebuilt_sentence+"."
    return(rebuilt_sentence)

def passive_to_active_neg(sentence):
    string_split = sentence.split()
    verb = string_split[2:4]
    if string_split[2:4] == ['not', 'shot']:
        verb = 'does not shoot'
    
    if string_split[2] == ['not', 'killed']:
        verb = 'does not kill'
        
    if string_split[2] == ['not', 'loved']:
        verb = 'does not love'
   
    
    arg0 = string_split[-1][:-1]
    arg1 = string_split[0]
    rebuilt_sent = arg0+" "+verb+ " "+arg1+"."
    return rebuilt_sent

In [196]:
srl_predictor = load_predictor('structured-prediction-srl-bert')

lerc is not a registered model.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [197]:
passive = make_list_shoot_passive(nsamples)
df = pd.DataFrame(passive)

df['active'] = df[0].apply(passive_to_active)
df['tags_act'] = df['active'].apply(get_tags_v0)
df['descr_act'] = df['active'].apply(get_descr_v0)
df['tags_pass'] = df[0].apply(get_tags_v1)
df['descr_pass'] = df[0].apply(get_descr_v1)
df['first_tag_act'] = df['tags_act'].apply(find_first_arg)
df['final_min_one_pass'] = df['tags_pass'].apply(find_final_min_one_arg)
df['first_tag_act'] = df['first_tag_act'].str[2:]
df['final_min_one_pass'] = df['final_min_one_pass'].str[2:]
df['arg0_match'] = df['final_min_one_pass']==df['first_tag_act']


df[0].to_csv("../datasets/voice/passive.txt", header=None, index=False)
df['active'].to_csv("../datasets/voice/active.txt", header=None, index=False)
df.to_csv("../results/voice/active-passive_arg0_bert.csv")
error_arg0_bert = len(df[df['arg0_match']==False])/len(df)

In [198]:
passive_neg = make_list_shoot_passive_neg(nsamples)
df = pd.DataFrame(passive_neg)
df['active'] = df[0].apply(passive_to_active_neg)
df['tags_act'] = df['active'].apply(get_tags_v1)
df['descr_act'] = df['active'].apply(get_descr_v1)
df['tags_pass'] = df[0].apply(get_tags_v1)
df['descr_pass'] = df[0].apply(get_descr_v1)
df['first_tag_act'] = df['tags_act'].apply(find_first_arg)
df['final_min_one_pass'] = df['tags_pass'].apply(find_final_min_one_arg)
df['first_tag_act'] = df['first_tag_act'].str[2:]
df['final_min_one_pass'] = df['final_min_one_pass'].str[2:]
df['arg0_match'] = df['final_min_one_pass']==df['first_tag_act']


df[0].to_csv("../datasets/voice/passive_neg.txt", header=None, index=False)
df['active'].to_csv("../datasets/voice/active_neg.txt", header=None, index=False)
df.to_csv("../results/voice/active-passive_arg0_neg_bert.csv")
error_arg0_neg_bert = len(df[df['arg0_match']==False])/len(df)

In [199]:
srl_predictor = load_predictor('structured-prediction-srl')

lerc is not a registered model.


In [200]:
passive = make_list_shoot_passive(nsamples)
df = pd.DataFrame(passive)

df['active'] = df[0].apply(passive_to_active)
df['tags_act'] = df['active'].apply(get_tags_v0)
df['descr_act'] = df['active'].apply(get_descr_v0)
df['tags_pass'] = df[0].apply(get_tags_v1)
df['descr_pass'] = df[0].apply(get_descr_v1)
df['first_tag_act'] = df['tags_act'].apply(find_first_arg)
df['final_min_one_pass'] = df['tags_pass'].apply(find_final_min_one_arg)
df['first_tag_act'] = df['first_tag_act'].str[2:]
df['final_min_one_pass'] = df['final_min_one_pass'].str[2:]
df['arg0_match'] = df['final_min_one_pass']==df['first_tag_act']


df[0].to_csv("../datasets/voice/passive.txt", header=None, index=False)
df['active'].to_csv("../datasets/voice/active.txt", header=None, index=False)

df.to_csv("../results/voice/active-passive_arg0_bilstm.csv")
error_arg0_bilstm = len(df[df['arg0_match']==False])/len(df)

In [201]:
passive_neg = make_list_shoot_passive_neg(nsamples)
df = pd.DataFrame(passive_neg)
df['active'] = df[0].apply(passive_to_active_neg)
df['tags_act'] = df['active'].apply(get_tags_v1)
df['descr_act'] = df['active'].apply(get_descr_v1)
df['tags_pass'] = df[0].apply(get_tags_v1)
df['descr_pass'] = df[0].apply(get_descr_v1)
df['first_tag_act'] = df['tags_act'].apply(find_first_arg)
df['final_min_one_pass'] = df['tags_pass'].apply(find_final_min_one_arg)
df['first_tag_act'] = df['first_tag_act'].str[2:]
df['final_min_one_pass'] = df['final_min_one_pass'].str[2:]
df['arg0_match'] = df['final_min_one_pass']==df['first_tag_act']

df.to_csv("../results/voice/active-passive_arg0_neg_bilstm.csv")
error_arg0_neg_bilstm = len(df[df['arg0_match']==False])/len(df)

In [202]:
df[df['arg0_match']==False]

,0,active,tags_act,descr_act,tags_pass,descr_pass,first_tag_act,final_min_one_pass,arg0_match
10,Patricia is not shot by Kathy.,Kathy does not shoot Patricia.,"[B-ARGM-MNR, O, B-ARGM-NEG, B-V, B-ARG1, O]",[ARGM-MNR: Kathy] does [ARGM-NEG: not] [V: sho...,"[B-ARG1, O, B-ARGM-NEG, B-V, B-ARG0, I-ARG0, O]",[ARG1: Patricia] is [ARGM-NEG: not] [V: shot] ...,ARGM-MNR,ARG0,False
43,George is not shot by Dan.,Dan does not shoot George.,"[B-ARGM-DIS, O, B-ARGM-NEG, B-V, B-ARG1, O]",[ARGM-DIS: Dan] does [ARGM-NEG: not] [V: shoot...,"[B-ARG1, O, B-ARGM-NEG, B-V, B-ARG0, I-ARG0, O]",[ARG1: George] is [ARGM-NEG: not] [V: shot] [A...,ARGM-DIS,ARG0,False
97,Mark is not shot by Ron.,Ron does not shoot Mark.,"[B-ARG0, O, B-ARGM-NEG, B-V, B-ARG1, O]",[ARG0: Ron] does [ARGM-NEG: not] [V: shoot] [A...,"[B-ARG1, O, B-ARGM-NEG, B-V, B-ARGM-TMP, I-ARG...",[ARG1: Mark] is [ARGM-NEG: not] [V: shot] [ARG...,ARG0,ARGM-TMP,False


In [206]:
print("BERT")
print("arg0: ",error_arg0_bert*100)
print("arg0neg: ",error_arg0_bert_neg)
print()
print("bilstm")
print("arg0: ",error_arg0_bilstm*100)
print("arg0neg: ",error_arg0_neg_bilstm*100)

BERT
arg0:  8.0
arg0neg:  0.0

bilstm
arg0:  22.22222222222222
arg0neg:  3.0
